In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv('RawData/2012-2013 Solar home electricity data v2.csv', skiprows=1)
df.head()

,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,...,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00,Row Quality
0,1,3.78,2076,CL,1/07/2012,1.250,1.250,1.250,1.263,0.131,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.081,NaN
1,1,3.78,2076,GC,1/07/2012,0.855,0.786,0.604,0.544,0.597,...,0.374,0.447,0.549,0.136,0.288,0.181,0.651,0.090,0.068,NaN
2,1,3.78,2076,GG,1/07/2012,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
3,1,3.78,2076,CL,2/07/2012,1.250,1.250,1.125,0.000,0.925,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.069,NaN
4,1,3.78,2076,GC,2/07/2012,0.309,0.082,0.059,0.097,0.290,...,0.353,0.464,0.229,0.811,0.222,0.306,1.034,0.136,0.067,NaN


In [38]:
filtered_customer_id = [2, 13, 14, 20, 33, 35, 38, 39, 56,
                        69, 73, 74, 75, 82, 87, 88, 101, 104,
                        106, 109, 110, 119, 124, 130, 137, 141, 144,
                        152, 153, 157, 161, 169, 176, 184, 188, 189,
                        193, 201, 202, 204, 206, 207, 210, 211, 212,
                        214, 218, 244, 246, 253, 256, 273, 276, 297 
                       ]
df = df[df.Customer.isin( filtered_customer_id)]


df_GG =  df[df['Consumption Category'] == 'GG']
df_GC =  df[df['Consumption Category'] == 'GC']
df_CL =  df[df['Consumption Category'] == 'CL']
half_hour = df_GG.loc[:,'0:30':'0:00'].columns
timecodes = []
for i in df_GG.date:
    for j in half_hour:
        timecodes.append(i+' ' + j)

power_df = pd.DataFrame(timecodes)
power_df.rename(columns={0:'Timestamp'}, inplace=True)
power_df.insert(0,'Customer', value='NA')
power_df.insert(1,'Postcode', value='NA')

power_df.insert(3,'Solar_Production_kWh',value='NA')
power_df.insert(4,'Energy_Consumption_kWh',value='NA')
power_df.insert(5,'Heating_Consumption_kWh',value='NA')

counter_GG = 0
counter_GC = 0
counter_CL = 0
for i in df.Customer.unique():
   
    one_cust_GG = df_GG[df_GG.Customer == i]
    one_cust_GC = df_GC[df_GC.Customer == i]
    one_cust_CL = df_CL[df_CL.Customer == i]
    post = one_cust_GG.Postcode.unique()
    cust = one_cust_GG.Customer.unique()
    
    
    for index , row in one_cust_GG.iterrows():
        gg = row.T.loc['0:30':'0:00']
        for j in range(48):
            power_df.loc[counter_GG, 'Customer'] = cust[0]
            power_df.loc[counter_GG, 'Postcode'] = post[0]
            power_df.loc[counter_GG, 'Solar_Production_kWh'] = gg.values[j]
            counter_GG +=1
    
    for index , row in one_cust_GC.iterrows():
        gc = row.T.loc['0:30':'0:00']
        for j in range(48):
            power_df.loc[counter_GC, 'Energy_Consumption_kWh'] = gc.values[j]
            counter_GC +=1
   
    
    for index , row in one_cust_CL.iterrows():
        cl = row.T.loc['0:30':'0:00']
        for j in range(48):
            power_df.loc[counter_CL, 'Heating_Consumption_kWh'] = cl.values[j]
            counter_CL +=1
  
    
power_df.Timestamp = pd.to_datetime(power_df.Timestamp, format='%d/%m/%Y %H:%M')
# power_df.Power_production_kWh = power_df.Power_production_kWh.astype('float64')
# power_df.Power_Consumption_kWh = power_df.Power_Consumption_kWh.astype('float64')
power_df.head()

,Customer,Postcode,Timestamp,Solar_Production_kWh,Energy_Consumption_kWh,Heating_Consumption_kWh
0,2,2259,2012-07-01 00:30:00,0,0.167,0
1,2,2259,2012-07-01 01:00:00,0,0.108,0
2,2,2259,2012-07-01 01:30:00,0,0.119,0
3,2,2259,2012-07-01 02:00:00,0,0.15,0
4,2,2259,2012-07-01 02:30:00,0,0.109,0


In [39]:
power_df.insert(6,'Price_AUD/MWh',value='NA')
power_df.insert(7,'Total_Demand_MWh',value='NA')
power_df.insert(8,'Latitude', value='NA')
power_df.insert(9,'Longitude', value='NA')

power_df.head()

,Customer,Postcode,Timestamp,Solar_Production_kWh,Energy_Consumption_kWh,Heating_Consumption_kWh,Price_AUD/MWh,Total_Demand_MWh,Latitude,Longitude
0,2,2259,2012-07-01 00:30:00,0,0.167,0,NA,NA,NA,NA
1,2,2259,2012-07-01 01:00:00,0,0.108,0,NA,NA,NA,NA
2,2,2259,2012-07-01 01:30:00,0,0.119,0,NA,NA,NA,NA
3,2,2259,2012-07-01 02:00:00,0,0.15,0,NA,NA,NA,NA
4,2,2259,2012-07-01 02:30:00,0,0.109,0,NA,NA,NA,NA


# Price and Electricty demand Data for NSW, Australia  (1 July 2010 to 30 June 2013) 

In [40]:
import glob

path = r'RawData/PriceData' 
all_files = glob.glob(path + "/*.csv")
files = []
for filename in all_files:
    p_d_df = pd.read_csv(filename, index_col=None, header=0)
    files.append(p_d_df)

price_demand_df = pd.concat(files, axis=0, ignore_index=True)
price_demand_df.SETTLEMENTDATE = pd.to_datetime(price_demand_df.SETTLEMENTDATE)
price_demand_df.rename(columns={'SETTLEMENTDATE': 'Timestamp','RRP': 'Price_AUD/MWh', 'TOTALDEMAND':'Total_Demand_MWh' }, inplace=True)
price_demand_df.drop('PERIODTYPE', axis=1, inplace=True)
price_demand_df.drop([len(price_demand_df)-1], axis=0, inplace=True)
price_demand_df.head()

,REGION,Timestamp,Total_Demand_MWh,Price_AUD/MWh
0,NSW1,2012-07-01 00:30:00,8097.93,57.04
1,NSW1,2012-07-01 01:00:00,7852.57,53.69
2,NSW1,2012-07-01 01:30:00,7632.28,51.94
3,NSW1,2012-07-01 02:00:00,7384.70,52.13
4,NSW1,2012-07-01 02:30:00,7112.43,47.44


# Location data based on Post code of users in Australia

In [41]:
post_df = pd.read_csv('RawData/Australian_Post_Codes_Lat_Lon.csv')
post_df.rename(columns={'postcode': 'Postcode', 'lat': 'Latitude', 'lon': 'Longitude' }, inplace=True)
post_df.drop(['dc','type','suburb', 'state' ], axis=1, inplace=True)
post_df.head()

,Postcode,Latitude,Longitude
0,200,-35.277272,149.117136
1,221,-35.201372,149.095065
2,800,-12.801028,130.955789
3,801,-12.801028,130.955789
4,804,-12.432181,130.843310


# Merging Location data onto power_df Dataframe

In [42]:
lat = post_df.set_index('Postcode').Latitude.to_dict()
lon = post_df.set_index('Postcode').Longitude.to_dict()
power_df['Latitude'] = power_df.Postcode.apply(lambda x: lat[x])
power_df['Longitude'] = power_df.Postcode.apply(lambda x: lon[x])
power_df

,Customer,Postcode,Timestamp,Solar_Production_kWh,Energy_Consumption_kWh,Heating_Consumption_kWh,Price_AUD/MWh,Total_Demand_MWh,Latitude,Longitude
0,2,2259,2012-07-01 00:30:00,0,0.167,0,NA,NA,-33.222529,151.277495
1,2,2259,2012-07-01 01:00:00,0,0.108,0,NA,NA,-33.222529,151.277495
2,2,2259,2012-07-01 01:30:00,0,0.119,0,NA,NA,-33.222529,151.277495
3,2,2259,2012-07-01 02:00:00,0,0.15,0,NA,NA,-33.222529,151.277495
4,2,2259,2012-07-01 02:30:00,0,0.109,0,NA,NA,-33.222529,151.277495
...,...,...,...,...,...,...,...,...,...,...
942187,297,2261,2013-06-30 22:00:00,0,0.167,NA,NA,NA,-33.362679,151.447302
942188,297,2261,2013-06-30 22:30:00,0,0.179,NA,NA,NA,-33.362679,151.447302
942189,297,2261,2013-06-30 23:00:00,0,0.181,NA,NA,NA,-33.362679,151.447302
942190,297,2261,2013-06-30 23:30:00,0,0.171,NA,NA,NA,-33.362679,151.447302


# Merging Price and Demand Data power_df Dataframe

In [43]:
temp_df = power_df.copy()
temp_df['Total_Demand_MWh'] = temp_df.Timestamp.map(price_demand_df.set_index('Timestamp')['Total_Demand_MWh'])
temp_df['Price_AUD/MWh'] = temp_df.Timestamp.map(price_demand_df.set_index('Timestamp')['Price_AUD/MWh'])
power_df['Total_Demand_MWh'] = temp_df['Total_Demand_MWh']
power_df['Price_AUD/MWh'] = temp_df['Price_AUD/MWh']
power_df

,Customer,Postcode,Timestamp,Solar_Production_kWh,Energy_Consumption_kWh,Heating_Consumption_kWh,Price_AUD/MWh,Total_Demand_MWh,Latitude,Longitude
0,2,2259,2012-07-01 00:30:00,0,0.167,0,57.04,8097.93,-33.222529,151.277495
1,2,2259,2012-07-01 01:00:00,0,0.108,0,53.69,7852.57,-33.222529,151.277495
2,2,2259,2012-07-01 01:30:00,0,0.119,0,51.94,7632.28,-33.222529,151.277495
3,2,2259,2012-07-01 02:00:00,0,0.15,0,52.13,7384.70,-33.222529,151.277495
4,2,2259,2012-07-01 02:30:00,0,0.109,0,47.44,7112.43,-33.222529,151.277495
...,...,...,...,...,...,...,...,...,...,...
942187,297,2261,2013-06-30 22:00:00,0,0.167,NA,50.68,8761.76,-33.362679,151.447302
942188,297,2261,2013-06-30 22:30:00,0,0.179,NA,55.80,8620.94,-33.362679,151.447302
942189,297,2261,2013-06-30 23:00:00,0,0.181,NA,53.27,8358.81,-33.362679,151.447302
942190,297,2261,2013-06-30 23:30:00,0,0.171,NA,53.69,8154.82,-33.362679,151.447302


In [44]:
power_df.describe()

,Price_AUD/MWh,Total_Demand_MWh,Latitude,Longitude
count,942138.000000,942138.000000,942192.000000,942192.000000
mean,55.106871,8098.187366,-33.217984,151.448070
std,12.911398,1236.770773,0.321609,0.231941
min,-59.280000,5074.630000,-34.048156,151.041806
25%,50.070000,7182.560000,-33.362679,151.277495
50%,52.800000,8183.370000,-33.209738,151.480970
75%,56.060000,8912.290000,-32.950811,151.690553
max,317.970000,13787.850000,-32.696956,151.777382


In [9]:
power_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5252112 entries, 0 to 5252111
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Customer               object        
 1   Postcode               object        
 2   Timestamp              datetime64[ns]
 3   Power_production_kWh   float64       
 4   Power_Consumption_kWh  float64       
 5   Latitude               float64       
 6   Longitude              float64       
 7   Total_Demand_MWh       float64       
 8   Price_AUD/MWh          float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 360.6+ MB


In [45]:
power_df.describe()

,Price_AUD/MWh,Total_Demand_MWh,Latitude,Longitude
count,942138.000000,942138.000000,942192.000000,942192.000000
mean,55.106871,8098.187366,-33.217984,151.448070
std,12.911398,1236.770773,0.321609,0.231941
min,-59.280000,5074.630000,-34.048156,151.041806
25%,50.070000,7182.560000,-33.362679,151.277495
50%,52.800000,8183.370000,-33.209738,151.480970
75%,56.060000,8912.290000,-32.950811,151.690553
max,317.970000,13787.850000,-32.696956,151.777382


In [46]:
power_df.set_index('Customer').to_csv('Energy_Dataset.csv')